In [8]:
from line_solver import *

import numpy as np

GlobalConstants.setVerbose(VerboseLevel.STD)

In [9]:
model = Network('model')

n = 5  # number of items
m = 2  # cache capacity

source = Source(model, 'Source')
cacheNode = Cache(model, 'Cache', n, m, ReplacementStrategy.RR)
sink = Sink(model, 'Sink')

jobClass = OpenClass(model, 'InitClass', 0)
hitClass = OpenClass(model, 'HitClass', 0)
missClass = OpenClass(model, 'MissClass', 0)

source.setArrival(jobClass, Exp(2))

alpha = 1.4
pAccess = Zipf(alpha, n)  # Zipf-like item references
cacheNode.setRead(jobClass, pAccess)

cacheNode.setHitClass(jobClass, hitClass)
cacheNode.setMissClass(jobClass, missClass)

P = model.initRoutingMatrix()
P.set(jobClass, jobClass, source, cacheNode, 1.0)
P.set(hitClass, hitClass, cacheNode, sink, 1.0)
P.set(missClass, missClass, cacheNode, sink, 1.0)

model.link(P)

In [10]:
solver = np.empty(4, dtype=object)
AvgTable = np.empty(4, dtype=object)

#TODO: CTMC fails
#solver[0] = SolverCTMC(model, "keep",False, "cutoff", 1, "seed",1)
#AvgTable[0] = solver[0].getAvgNodeTable()

#model.reset()
#TODO: SSA fails
#solver[1] = SolverSSA(model, "samples", 100000, "verbose", True, "method","serial", "seed", 1)
#AvgTable[1] = solver[1].getAvgNodeTable()

model.reset()
solver[2] = SolverMVA(model)
AvgTable[2] = solver[2].getAvgNodeTable()

model.reset()
#TODO: this does not seem to execute rayint
solver[3] = SolverNC(model)
AvgTable[3] = solver[3].getAvgNodeTable()

hitRatio = cacheNode.getHitRatio(); print(hitRatio)
missRatio = cacheNode.getMissRatio(); print(missRatio)

MVA [method: default(default), lang: java, env: 21.0.7] completed in 0.001000s.
     Node   JobClass  QLen  Util  RespT  ResidT    ArvR    Tput
0  Source  InitClass   0.0   0.0    0.0     0.0  0.0000  2.0000
3   Cache  InitClass   0.0   0.0    0.0     0.0  2.0000  0.0000
4   Cache   HitClass   0.0   0.0    0.0     0.0  0.0000  1.1127
5   Cache  MissClass   0.0   0.0    0.0     0.0  0.0000  0.8873
7    Sink   HitClass   0.0   0.0    0.0     0.0  1.1127  0.0000
8    Sink  MissClass   0.0   0.0    0.0     0.0  0.8873  0.0000
NC [method: default(exact), lang: java, env: 21.0.7] completed in 0.001000s.
     Node   JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
0  Source  InitClass   0.0   0.0    0.0     0.0   0.0   2.0
3   Cache  InitClass   0.0   0.0    0.0     0.0   2.0   0.0
4   Cache   HitClass   0.0   0.0    0.0     0.0   0.0   1.0
5   Cache  MissClass   0.0   0.0    0.0     0.0   0.0   1.0
7    Sink   HitClass   0.0   0.0    0.0     0.0   1.0   0.0
8    Sink  MissClass   0.0   0.0   